# Indices Calculation

In [2]:
import calendar
import numpy as np
import openapi_client as dbitApi
import pandas as pd

from datetime import datetime

In [14]:
def format_datetime_to_expiry(date):
    return datetime.strftime(date, '%-d%b%y').upper()

def get_near_next_terms(now):
    c = calendar.Calendar(firstweekday=calendar.MONDAY)
    
    this_month_cal = c.monthdatescalendar(now.year, now.month)
    this_fridays = [datetime(day.year, day.month, day.day, 8, 0, 0) 
                    for week in this_month_cal for day in week 
                    if day.weekday() == calendar.FRIDAY and day.month == now.month 
                    and datetime(day.year, day.month, day.day, 8, 0, 0) >= now]
    
    next_year = now.year if now.month < 12 else now.year + 1
    next_month = now.month + 1 if now.month < 12 else 1
    
    next_month_cal = c.monthdatescalendar(next_year, next_month)
    next_fridays = [datetime(day.year, day.month, day.day, 8, 0, 0) 
                    for week in next_month_cal for day in week 
                    if day.weekday() == calendar.FRIDAY and day.month == next_month 
                    and datetime(day.year, day.month, day.day, 8, 0, 0) >= now]
    
    fridays = this_fridays + next_fridays
    
    near_term, next_term = fridays[0], fridays[1]
        
    return (format_datetime_to_expiry(near_term), format_datetime_to_expiry(next_term), near_term, next_term)

def get_index(currency='BTC'):
    try:
        index_result = api.public_get_index_get(currency)['result'][currency]
        return index_result
    except dbitApi.exceptions.ApiException as e:
        print(e)
        #logger.exception('Exception when calling MarketDataApi->public_get_instruments_get!')
        exit()

def get_instruments_with_expiry(expiry, currency='BTC', kind='option', expired='false'):
    try:
        instrument_result = api.public_get_instruments_get(currency, kind=kind, expired=expired)['result']
        return [instrument['instrument_name'] for instrument in instrument_result if expiry in instrument['instrument_name']]
    except dbitApi.exceptions.ApiException as e:
        print(e)
        #logger.exception('Exception when calling MarketDataApi->public_get_instruments_get!')
        exit()

def get_ticker(instrument):
    try:
        instrument_result = api.public_ticker_get(instrument)['result']
        return instrument_result
    except dbitApi.exceptions.ApiException as e:
        print(e)
        #logger.exception('Exception when calling MarketDataApi->public_get_instruments_get!')
        exit()

def get_bids_asks(near_list, next_list):
    near_calls = dict()
    near_puts = dict()
    next_calls = dict()
    next_puts = dict()

    for instrument in near_list:
        data = get_ticker(instrument)
        best_bid, best_ask = data['best_bid_price'], data['best_ask_price']
        strike, cp = int(instrument.split('-')[2]), instrument.split('-')[3]

        if cp == 'C':
            near_calls[strike] = {'best_bid': best_bid, 'best_ask': best_ask}
        elif cp == 'P':
            near_puts[strike] = {'best_bid': best_bid, 'best_ask': best_ask}
        else:
            print(f'Error {instrument}')

    for instrument in next_list:
        data = get_ticker(instrument)
        best_bid, best_ask = data['best_bid_price'], data['best_ask_price']
        strike, cp = int(instrument.split('-')[2]), instrument.split('-')[3]

        if cp == 'C':
            next_calls[strike] = {'best_bid': best_bid, 'best_ask': best_ask}
        elif cp == 'P':
            next_puts[strike] = {'best_bid': best_bid, 'best_ask': best_ask}
        else:
            print(f'Error {instrument}')

    near_calls_df = pd.DataFrame.from_dict(near_calls, orient='index').sort_index()
    near_puts_df = pd.DataFrame.from_dict(near_puts, orient='index').sort_index()
    next_calls_df = pd.DataFrame.from_dict(next_calls, orient='index').sort_index()
    next_puts_df = pd.DataFrame.from_dict(next_puts, orient='index').sort_index()

    return near_calls_df, near_puts_df, next_calls_df, next_puts_df

In [15]:
api = dbitApi.MarketDataApi()

now = datetime.now()
near_expiry, next_expiry, near_datetime, next_datetime = get_near_next_terms(now)

print(near_expiry, next_expiry)

24APR20 1MAY20


In [16]:
near_instruments = get_instruments_with_expiry(near_expiry)
next_instruments = get_instruments_with_expiry(next_expiry)

near_calls_df, near_puts_df, next_calls_df, next_puts_df = get_bids_asks(near_instruments, next_instruments)

In [17]:
near_calls_df

,best_bid,best_ask
3000,0.5715,0.5945
3500,0.5000,0.5270
4000,0.4290,0.4600
4500,0.3580,0.3930
5000,0.2870,0.3260
5500,0.2160,0.2600
6000,0.1455,0.1960
6250,0.1125,0.1625
6500,0.0980,0.1220
6750,0.0710,0.0770


In [18]:
near_prices = pd.DataFrame(index=near_calls_df.index)
near_prices['call_price'] = (near_calls_df['best_bid'] + near_calls_df['best_ask']) / 2
near_prices['put_price'] = (near_puts_df['best_bid'] + near_puts_df['best_ask']) / 2
near_prices['abs_diff'] = abs(near_prices['call_price'] - near_prices['put_price'])

min_near_strike = near_prices['abs_diff'].idxmin()
min_near_diff = near_prices.loc[min_near_strike].abs_diff

next_prices = pd.DataFrame(index=next_calls_df.index)
next_prices['call_price'] = (next_calls_df['best_bid'] + next_calls_df['best_ask']) / 2
next_prices['put_price'] = (next_puts_df['best_bid'] + next_puts_df['best_ask']) / 2
next_prices['abs_diff'] = abs(next_prices['call_price'] - next_prices['put_price'])

min_next_strike = next_prices['abs_diff'].idxmin()
min_next_diff = next_prices.loc[min_next_strike].abs_diff

near_prices.head()

,call_price,put_price,abs_diff
3000,0.5830,0.00025,0.58275
3500,0.5135,0.00050,0.51300
4000,0.4445,0.00075,0.44375
4500,0.3755,0.00100,0.37450
5000,0.3065,0.00175,0.30475


In [167]:
const_mature_days = 7
R = 0

n1 = (near_datetime - now).total_seconds() / 60
n2 = (next_datetime - now).total_seconds() / 60
nY = 525600
n = const_mature_days * 24 * 60

t1 = n1/nY
t2 = n2/nY

# Compute forward prices and at-the-money strikes
f1 = min_near_strike + np.e**(R*t1) * min_near_diff
k0_1 = max([strike for strike in near_prices.index if strike <= min_near_strike])

f2 = min_next_strike + np.e**(R*t2) * min_next_diff
k0_2 = max([strike for strike in next_prices.index if strike <= min_next_strike])

print(k0_1, f1, k0_2, f2)

7250 7250.0085 7250 7250.0085


In [168]:
near_otm_puts_df = near_puts_df.loc[:k0_1][:-1]
near_otm_calls_df = near_calls_df.loc[k0_1:][1:]
next_otm_puts_df = next_puts_df.loc[:k0_2][:-1]
next_otm_calls_df = next_calls_df.loc[k0_2:][1:]

In [169]:
near_otm_puts_df

,best_bid,best_ask
3000,0.0000,0.0005
3500,0.0000,0.0010
4000,0.0005,0.0010
4500,0.0005,0.0015
5000,0.0015,0.0020
5500,0.0020,0.0025
6000,0.0040,0.0045
6250,0.0050,0.0065
6500,0.0085,0.0090
6750,0.0115,0.0135


In [170]:
near_otm_calls_df

,best_bid,best_ask
7500,0.0155,0.0175
7750,0.0090,0.0105
8000,0.0055,0.0070
8250,0.0035,0.0050
8500,0.0025,0.0040
9000,0.0015,0.0020
10000,0.0005,0.0015
11000,0.0000,0.0005
12000,0.0000,0.0005
13000,0.0000,0.0005


Exclude strikes following two consecutive bid prices and strikes with zero bids

In [171]:
near_otm_puts_df = near_otm_puts_df.sort_index(ascending=False)
near_otm_puts_df = near_otm_puts_df.assign(zero_bid=lambda df: (df['best_bid'] == 0).astype(int))
near_otm_puts_df['zero_bid_cumsum'] = near_otm_puts_df['zero_bid'].cumsum()
near_otm_puts_df = near_otm_puts_df[(near_otm_puts_df['zero_bid_cumsum'] <= 2) & (near_otm_puts_df['best_bid'] > 0)]

near_otm_puts_df

,best_bid,best_ask,zero_bid,zero_bid_cumsum
7000,0.0205,0.0215,0,0
6750,0.0115,0.0135,0,0
6500,0.0085,0.0090,0,0
6250,0.0050,0.0065,0,0
6000,0.0040,0.0045,0,0
5500,0.0020,0.0025,0,0
5000,0.0015,0.0020,0,0
4500,0.0005,0.0015,0,0
4000,0.0005,0.0010,0,0


In [172]:
near_otm_calls_df = near_otm_calls_df.assign(zero_bid=lambda df: (df['best_bid'] == 0).astype(int))
near_otm_calls_df['zero_bid_cumsum'] = near_otm_calls_df['zero_bid'].cumsum()
near_otm_calls_df = near_otm_calls_df[(near_otm_calls_df['zero_bid_cumsum'] <= 2) & (near_otm_calls_df['best_bid'] > 0)]

near_otm_calls_df

,best_bid,best_ask,zero_bid,zero_bid_cumsum
7500,0.0155,0.0175,0,0
7750,0.0090,0.0105,0,0
8000,0.0055,0.0070,0,0
8250,0.0035,0.0050,0,0
8500,0.0025,0.0040,0,0
9000,0.0015,0.0020,0,0
10000,0.0005,0.0015,0,0


In [173]:
next_otm_puts_df = next_otm_puts_df.sort_index(ascending=False)
next_otm_puts_df = next_otm_puts_df.assign(zero_bid=lambda df: (df['best_bid'] == 0).astype(int))
next_otm_puts_df['zero_bid_cumsum'] = next_otm_puts_df['zero_bid'].cumsum()
next_otm_puts_df = next_otm_puts_df[(next_otm_puts_df['zero_bid_cumsum'] <= 2) & (next_otm_puts_df['best_bid'] > 0)]

next_otm_calls_df = next_otm_calls_df.assign(zero_bid=lambda df: (df['best_bid'] == 0).astype(int))
next_otm_calls_df['zero_bid_cumsum'] = next_otm_calls_df['zero_bid'].cumsum()
next_otm_calls_df = next_otm_calls_df[(next_otm_calls_df['zero_bid_cumsum'] <= 2) & (next_otm_calls_df['best_bid'] > 0)]

In [174]:
next_otm_puts_df

,best_bid,best_ask,zero_bid,zero_bid_cumsum
7000,0.0375,0.0390,0,0
6750,0.0245,0.0285,0,0
6500,0.0180,0.0205,0,0
6250,0.0125,0.0155,0,0
6000,0.0090,0.0105,0,0
5750,0.0070,0.0080,0,0
5500,0.0045,0.0060,0,0
5250,0.0035,0.0050,0,0


In [175]:
next_otm_calls_df

,best_bid,best_ask,zero_bid,zero_bid_cumsum
7500,0.0300,0.0335,0,0
7750,0.0205,0.0240,0,0
8000,0.0135,0.0170,0,0
8250,0.0110,0.0130,0,0
8500,0.0075,0.0100,0,0
8750,0.0055,0.0080,0,0
9000,0.0045,0.0065,0,0


In [176]:
near_calc_strikes_df = pd.DataFrame(index=near_prices.index)
near_calc_strikes_df['price'] = (near_otm_puts_df['best_bid'] + near_otm_puts_df['best_ask']) / 2
near_calc_strikes_df['price'] = near_calc_strikes_df.price.combine_first((near_otm_calls_df['best_bid'] + near_otm_calls_df['best_ask']) / 2)
near_calc_strikes_df.at[k0_1] = (near_prices.loc[k0_1].call_price + near_prices.loc[k0_1].put_price) / 2

In [177]:
near_calc_strikes_df = near_calc_strikes_df.dropna()
near_calc_strikes_df

,price
4000,0.00075
4500,0.00100
5000,0.00175
5500,0.00225
6000,0.00425
6250,0.00575
6500,0.00875
6750,0.01250
7000,0.02100
7250,0.03250


In [178]:
next_calc_strikes_df = pd.DataFrame(index=next_prices.index)
next_calc_strikes_df['price'] = (next_otm_puts_df['best_bid'] + next_otm_puts_df['best_ask']) / 2
next_calc_strikes_df['price'] = next_calc_strikes_df.price.combine_first((next_otm_calls_df['best_bid'] + next_otm_calls_df['best_ask']) / 2)
next_calc_strikes_df.at[k0_2] = (next_prices.loc[k0_2].call_price + next_prices.loc[k0_2].put_price) / 2

In [179]:
next_calc_strikes_df = next_calc_strikes_df.dropna()
next_calc_strikes_df

,price
5250,0.00425
5500,0.00525
5750,0.00750
6000,0.00975
6250,0.01400
6500,0.01925
6750,0.02650
7000,0.03825
7250,0.04900
7500,0.03175


In [181]:
near_sum = 0
for i in range(len(near_calc_strikes_df)):
    row = near_calc_strikes_df.iloc[i]
    if i == 0:
        deltaKi = near_calc_strikes_df.iloc[i+1].name - row.name
    elif i == len(near_calc_strikes_df) - 1:
        deltaKi = row.name - near_calc_strikes_df.iloc[i-1].name
    else:
        deltaKi = (near_calc_strikes_df.iloc[i+1].name + near_calc_strikes_df.iloc[i-1].name) / 2

    near_sum += deltaKi/(row.name ** 2) * np.e**(R*t1) * row.price
    
next_sum = 0
for i in range(len(next_calc_strikes_df)):
    row = next_calc_strikes_df.iloc[i]
    if i == 0:
        deltaKi = next_calc_strikes_df.iloc[i+1].name - row.name
    elif i == len(next_calc_strikes_df) - 1:
        deltaKi = row.name - next_calc_strikes_df.iloc[i-1].name
    else:
        deltaKi = (next_calc_strikes_df.iloc[i+1].name + next_calc_strikes_df.iloc[i-1].name) / 2
    
    next_sum += deltaKi/(row.name ** 2) * np.e**(R*t2) * row.price
    
sigma1 = ((2/t1) * near_sum) - (1/t1)*((f1/k0_1 - 1)**2)
sigma2 = ((2/t2) * next_sum) - (1/t2)*((f2/k0_2 - 1)**2)

print(sigma1, sigma2)

0.002433324965318429 0.0021766365185696517


In [187]:
XVBT = 100 * np.sqrt((t1*(sigma1**2)*((n2-n)/(n2-n1)))+(t2*(sigma2**2)*((n-n1)/(n2-n1)))*(nY/n))

In [189]:
XVBT

0.13353236688855782